In [1]:
from glob import glob
import os

from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.datasets import build_dataloader
from mmdet.apis import single_gpu_test
from mmcv.parallel import MMDataParallel

In [2]:
!mkdir checkpoints
!wget -c http://download.openmmlab.com/mmdetection/v2.0/yolo/yolov3_d53_mstrain-608_273e_coco/yolov3_d53_mstrain-608_273e_coco-139f5633.pth \
      -O checkpoints/yolov3_d53_mstrain-608_273e_coco-139f5633.pth

mkdir: cannot create directory ‘checkpoints’: File exists
--2021-11-03 14:52:37--  http://download.openmmlab.com/mmdetection/v2.0/yolo/yolov3_d53_mstrain-608_273e_coco/yolov3_d53_mstrain-608_273e_coco-139f5633.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 47.252.96.35
Connecting to download.openmmlab.com (download.openmmlab.com)|47.252.96.35|:80... connected.
HTTP request sent, awaiting response... 200 OK

    The file is already fully retrieved; nothing to do.



In [3]:
from mmdet.apis import inference_detector, init_detector, show_result_pyplot

config = 'configs/yolo/yolov3_d53_mstrain-608_273e_coco.py'
checkpoint = 'checkpoints/yolov3_d53_mstrain-608_273e_coco-139f5633.pth'
model = init_detector(config, checkpoint, device='cuda:0')

Use load_from_local loader


In [4]:
def create_dataset(folder):
  cfg = Config.fromfile('./configs/yolo/yolov3_d53_mstrain-608_273e_coco.py')
  cfg.dataset_type = 'CocoDataset'
  PREFIX = folder
  cfg.data.test.img_prefix = folder
  cfg.data.test.ann_file = os.path.join(PREFIX, 'new_test.json')
  cfg.data.test.type = 'CocoDataset'
  dataset = build_dataset(cfg.data.test)
  #dataset.CLASSES = classes
  #dataset.cat_ids = list(range(len(dataset.CLASSES)))
  data_loader = build_dataloader(dataset, samples_per_gpu=1, workers_per_gpu=2, dist=False, shuffle=False)
  return dataset, data_loader

In [5]:
!mkdir out

mkdir: cannot create directory ‘out’: File exists


In [6]:
dataset, data_loader = create_dataset('/home/franz/RTTS/ds/enhanced_retinex/')

loading annotations into memory...
Done (t=0.22s)
creating index...
index created!


In [7]:
model = MMDataParallel(model, device_ids=[0])

In [8]:
outputs = single_gpu_test(model, data_loader, True, './out', 0.3)

[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 4320/4320, 9.1 task/s, elapsed: 477s, ETA:     0s

In [9]:
dataset.cat_ids.append(91)

In [10]:
metric = dataset.evaluate(outputs, metric='bbox')


Evaluating bbox...
Loading and preparing results...
DONE (t=0.19s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=41.13s).
Accumulating evaluation results...
DONE (t=4.03s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.296
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.524
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.298
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.105
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.283
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.411
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.374
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.374
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.374
 Average Recall     (AR) @[ IoU=0.50:0.95